In [ ]:
import meow as mw
import jax.numpy as np
import matplotlib.pyplot as plt

## Preparation
let's perform a mode simulation

In [ ]:
length = 10.0
box = mw.Box(
    x_min=-0.22,
    x_max=0.22,
    y_min=0,
    y_max=0.22,
    z_min=0.0,
    z_max=length,
)

In [ ]:
struct = mw.Structure(material=mw.silicon, geometry=box)

In [ ]:
env = mw.Environment(wl=1.55, T=25.0)

In [ ]:
cells = mw.create_cells(
    structures=[struct],
    mesh=mw.Mesh2d(
        x=np.linspace(-1, 1, 101),
        y=np.linspace(-1, 1, 101),
    ),
    Ls=[length / 3] * 3,
)
css = [mw.CrossSection.from_cell(cell=cell, env=env) for cell in cells]

In [ ]:
modes = [mw.compute_modes(cs, num_modes=3) for cs in css]

In [ ]:
modes_ = modes.copy()

In [ ]:
modes = modes_.copy()

In [ ]:
modes[1] = modes[1][:-1]
modes[1] = [m * complex(np.exp(0.4j)) for m in modes[1]]

In [ ]:
from meow.eme.propagate import propagate_modes

In [ ]:
z = np.linspace(0, length, 300)
y = 0.5

ex_l = np.zeros(len(modes[0]))
ex_l = ex_l.at[0].set(1)
ex_r = np.zeros(len(modes[-1]))
ex_r = ex_r.at[0].set(0.3)

E, x = propagate_modes(modes, cells, ex_l, ex_r, y, z)

X, Y = np.meshgrid(z, x)
lim = np.quantile(np.abs(E), 0.98)
plt.pcolormesh(X, Y, np.abs(E).T, shading="nearest", cmap="magma", vmin=0, vmax=lim)